In [ ]:
import time
import pyspark
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel

In [ ]:
class Power:
    def __init__(self, p):
        self.p = p
        time.sleep(2)
        
    def applyPower(self, x):
        return x**self.p
        
# map
def power_map(num):
    # apply num ** 5
    c = Power(5)
    return c.applyPower(num)

In [ ]:
conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')
numbers = sc.textFile("../Data/numbers.txt", 5).map(lambda x : int(x))

In [ ]:
sc

## Without Persisting or Caching

In [ ]:
start = time.time()

print("first ", time.time() - start)

start = time.time()

print("second ", time.time() - start)

## With Persisting/Caching

In [ ]:
start = time.time()

print("first ", time.time() - start)  # trigger re-evaluation

In [ ]:
start = time.time()

print("second ", time.time() - start)  # doesn't trigger re-evaluation

## What happens?

In [ ]:
.persist(StorageLevel.MEMORY_AND_DISK)

## How to solve it?

## Reading/writing from disk

In [ ]:
powered_num.persist() #update
start = time.time()

print("first ", time.time() - start)  # trigger reevaluation

start = time.time()

print("second ", time.time() - start)  # doesn't trigger reevaluation

In [ ]:
sc.stop()